### 導入套件 + 設定路徑

In [ ]:
import pandas as pd
from pathlib import Path
import glob
import datetime
import re
import os
import numpy as np
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from datetime import date
import math


In [ ]:
# home and time
home = Path.home()
todaystr = date.today().strftime('%Y-%m-%d')

# set up concat directories
targetFolder = Path(home, 'HP Inc','GPSTW SOP - 2021 日新','Project team','Upload folder ( for buyer update )')
FD_folder = Path(targetFolder, "FD_today","*.xlsx")
shortage_folder = Path(targetFolder ,"shortage_today","*.xlsx")
PNbasedDetail_folder = Path(targetFolder ,"PNbasedDetail_today","*.xlsx")

### FD 合併以及整理

#### 合併檔案

In [ ]:
# concat FD
FD_files = []
for FD_file in glob.glob(str(Path(FD_folder))):
    print(FD_file)
    FD = pd.read_excel(FD_file)
    FD_files.append(FD)
FD_all = pd.concat(FD_files)


In [ ]:
# remember inplace = True
FD_all['len_FV'] = FD_all['FV'].str.len()
FD_all['len_platform'] = FD_all['Platform'].str.len()
FD_all = FD_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
FD_all.reset_index( drop = True, inplace = True )

#### 將max len放到前方

In [ ]:
# Index where B is longest
FD_idmax = FD_all['len_platform'].max()
df_max = FD_all.loc[FD_all['len_platform'] == FD_idmax]
df_max_to_add_FD = df_max.head(1)

In [ ]:
# isin用在list
# drop the max len row
FD_all = FD_all.drop([FD_all.index[df_max_to_add_FD.index.values[0]]]).reset_index( drop = True )

In [ ]:
# concat and put the max len row on the top
FD_concat = [df_max_to_add_FD,FD_all]
FD_output = pd.concat(FD_concat)
FD_output.reset_index( drop = True , inplace = True)

In [ ]:
# cut more than 500
FD_output['Platform'] = FD_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
FD_output = FD_output.drop( columns= ['len_FV','len_platform'])
FD_output['Item'] = FD_output['Item'].astype(str)
FD_output

### Shortage 合併以及整理

#### 合併檔案

In [ ]:
# concat shortage
shortage_files = []
for shortage_file in glob.glob(str(Path(shortage_folder))):
    print(shortage_file)
    shortage = pd.read_excel(shortage_file)
    shortage_files.append(shortage)
shortage_all = pd.concat(shortage_files)


In [ ]:
# create value to sort 
shortage_all['len_FV'] = shortage_all['FV'].str.len()
shortage_all['len_platform'] = shortage_all['Platform'].str.len()
shortage_all = shortage_all.sort_values(['len_FV','len_platform'] , ascending=[False,False])
shortage_all.reset_index( drop = True, inplace = True )

#### 將max len放到前面

In [ ]:
# find the max length 
shortage_idmax = shortage_all['len_platform'].max()
shortage_max = shortage_all.loc[shortage_all['len_platform'] == shortage_idmax]
df_max_to_add_shortage = shortage_max.head(1)
df_max_to_add_shortage

In [ ]:
# drop the max len row
shortage_all = shortage_all.drop([shortage_all.index[df_max_to_add_shortage.index.values[0]]]).reset_index( drop = True )

In [ ]:
# concat and put the max len row on the top
shortage_concat = [ df_max_to_add_shortage , shortage_all ]
shortage_output = pd.concat(shortage_concat)
shortage_output.reset_index( drop = True , inplace = True)

In [ ]:
# cut more than 500
shortage_output['Platform'] = shortage_output['Platform'].apply(lambda x: x[:450] if len(x) > 500 else x)
shortage_output = shortage_output.drop( columns= ['len_FV','len_platform'])
shortage_output['Item'] = shortage_output['Item'].astype(str)


### PNbasedDetail 合併及整理

In [ ]:
# concat PNbasedDetail
PNbasedDetail_files = []
for PNbasedDetail_file in glob.glob(str(Path(PNbasedDetail_folder))):
    print(PNbasedDetail_file)
    PNbasedDetail = pd.read_excel(PNbasedDetail_file)
    PNbasedDetail_files.append(PNbasedDetail)
PNbasedDetail_all = pd.concat(PNbasedDetail_files)

In [ ]:
# create list to sort 
sort_list = []
asc_list = []

try:
    PNbasedDetail_all['len_GPS Remark'] = PNbasedDetail_all['GPS Remark'].str.len()
    sort_list.append('len_GPS Remark')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column1'] = PNbasedDetail_all['ODM use column1'].str.len()
    sort_list.append('len_ODM use column1')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column2'] = PNbasedDetail_all['ODM use column2'].str.len()
    sort_list.append('len_ODM use column2')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column3'] = PNbasedDetail_all['ODM use column3'].str.len()
    sort_list.append('len_ODM use column3')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column4'] = PNbasedDetail_all['ODM use column4'].str.len()
    sort_list.append('len_ODM use column4')
    asc_list.append(True)
except:    
    pass
try:
    PNbasedDetail_all['len_ODM use column5'] = PNbasedDetail_all['ODM use column5'].str.len()
    sort_list.append('len_ODM use column5')
    asc_list.append(True)
except:    
    pass

In [ ]:
# sort value
PNbasedDetail_all = PNbasedDetail_all.sort_values( sort_list , ascending = asc_list )
PNbasedDetail_all.reset_index( drop=True , inplace=True ) 

In [ ]:
# find the max length & concat
PNbasedDetail_max_files = []

# a for loop to calculate all the len max
for i in range( 1 , len(sort_list) ):
    PNbasedDetail_idmax = PNbasedDetail_all [ sort_list[i] ].max()
    PNbasedDetail_max = PNbasedDetail_all.loc[ PNbasedDetail_all[ sort_list[i] ] == PNbasedDetail_idmax ]
    PNbasedDetail_max_files.append( PNbasedDetail_max.head(1) )
    df_max_to_add_PNbasedDetail_temp = pd.concat( PNbasedDetail_max_files )

# sometimes with duplicates 
df_max_to_add_PNbasedDetail = df_max_to_add_PNbasedDetail_temp.drop_duplicates()

In [ ]:
df_max_to_add_PNbasedDetail

In [ ]:
# drop the max len row
for i in range( 0, len(df_max_to_add_PNbasedDetail.index.values)):
    PNbasedDetail_all = PNbasedDetail_all.drop([PNbasedDetail_all.index[df_max_to_add_PNbasedDetail.index.values[i]]])

In [ ]:
# concat and put on the top
PNbasedDetail_concat_list = [ df_max_to_add_PNbasedDetail , PNbasedDetail_all ]
PNbasedDetail_output = pd.concat(PNbasedDetail_concat_list).reset_index( drop = True )
PNbasedDetail_output

In [ ]:
# cut more than 500
try:
    PNbasedDetail_output['GPS Remark'] = PNbasedDetail_output['GPS Remark'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column1'] = PNbasedDetail_output['ODM use column1'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column2'] = PNbasedDetail_output['ODM use column2'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column3'] = PNbasedDetail_output['ODM use column3'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column4'] = PNbasedDetail_output['ODM use column4'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass
try:
    PNbasedDetail_output['ODM use column5'] = PNbasedDetail_output['ODM use column5'].apply(lambda x: x[:450] if len(x) > 500 else x)
except:    
    pass


In [ ]:
# final step, drop calculate step and output
PNbasedDetail_output = PNbasedDetail_output.drop( columns = sort_list )
PNbasedDetail_output['Item'] = PNbasedDetail_output['Item'].astype(str)
PNbasedDetail_output

### FD、Shortage、PNbasedDetail 輸出檔案

In [ ]:
# apache airflow to upload SQL ( currently to desktop )
FD_output.to_excel(Path(home, 'Desktop', 'FD_all.xlsx'), index=False)
shortage_output.to_excel(Path(home, 'Desktop', 'Shortage_all.xlsx'), index=False)
PNbasedDetail_output.to_excel(Path(home, 'Desktop', 'PNbasedDetail_all.xlsx'), index=False)